In [ ]:
import ee
import geemap
import random

##uncomment if you need to authorize first before initalizing
ee.Authenticate()

# Initialize the Earth Engine module.
ee.Initialize()

In [4]:
def mask_s2_clouds(image):
    """Masks clouds in a Sentinel-2 image using the QA band."""
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

def calculate_ndvi(image):
    """Calculates NDVI for a Sentinel-2 image."""
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

def calculate_ndwi(image):
    """Calculates the Normalized Difference Water Index (NDWI) for a Sentinel-2 image."""
    ndwi = image.normalizedDifference(['B8', 'B11']).rename('NDWI')
    return image.addBands(ndwi)

def calculate_evi(image):
    """Calculates the Enhanced Vegetation Index (EVI) for a Sentinel-2 image."""
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename('EVI')
    return image.addBands(evi)

def calculate_savi(image):
    """Calculates the Soil Adjusted Vegetation Index (SAVI) for a Sentinel-2 image."""
    savi = image.expression(
        '(1 + L) * (NIR - RED) / (NIR + RED + L)', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'L': 0.5
        }).rename('SAVI')
    return image.addBands(savi)

def calculate_ndmi(image):
    """Calculates the Normalized Difference Moisture Index (NDMI) for a Sentinel-2 image."""
    ndmi = image.normalizedDifference(['B8', 'B11']).rename('NDMI')
    return image.addBands(ndmi)

def calculate_nbr(image):
    """Calculates the Normalized Burn Ratio (NBR) for a Sentinel-2 image."""
    nbr = image.normalizedDifference(['B8', 'B12']).rename('NBR')
    return image.addBands(nbr)

def is_land(lat, lon):
    """Check if the coordinates are over land using MODIS land cover data."""
    landcover = ee.Image('MODIS/006/MCD12Q1/2019_01_01').select('LC_Type1')
    point = ee.Geometry.Point([lon, lat])
    land_mask = landcover.reduceRegion(ee.Reducer.first(), point, 30).get('LC_Type1').getInfo()
    return land_mask != 0  # Returns True if it's land

def random_location_map():
    # Generate random latitude and longitude until it's over land.
    is_land_point = False
    while not is_land_point:
        lat = random.uniform(-90, 90)
        lon = random.uniform(-180, 180)
        is_land_point = is_land(lat, lon)
    
    # Prepare the dataset with cloud masking and index calculations.
    dataset = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
               .filterDate('2022-01-01', '2022-01-31')
               .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
               .map(mask_s2_clouds)
               .map(calculate_ndvi)
               .map(calculate_ndwi)
               .map(calculate_evi)
               .map(calculate_savi)
               .map(calculate_ndmi)
               .map(calculate_nbr))
    
    # Create a geemap object.
    m = geemap.Map()
    m.set_center(lon, lat, 12)  # Set the center to the random coordinates
    
    # Visualization parameters for each index
    ndvi_vis = {'min': 0, 'max': 1, 'palette': ['red', 'yellow', 'green']}
    ndwi_vis = {'min': 0, 'max': 1, 'palette': ['brown', 'blue']}
    evi_vis = {'min': 0, 'max': 1, 'palette': ['yellow', 'darkgreen']}
    savi_vis = {'min': 0, 'max': 1, 'palette': ['brown', 'green']}
    ndmi_vis = {'min': -1, 'max': 1, 'palette': ['yellow', 'blue']}
    nbr_vis = {'min': -1, 'max': 1, 'palette': ['green', 'black']}
    # RGB visualization parameters
    rgb_vis = {'min': 0.0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}
    
    # Add index layers to the map (assuming visualization parameters are defined)
    m.add_layer(dataset.median().select('NDVI'), ndvi_vis, 'NDVI')
    m.add_layer(dataset.median().select('NDWI'), ndwi_vis, 'NDWI')
    m.add_layer(dataset.median().select('EVI'), evi_vis, 'EVI')
    m.add_layer(dataset.median().select('SAVI'), savi_vis, 'SAVI')
    m.add_layer(dataset.median().select('NDMI'), ndmi_vis, 'NDMI')
    m.add_layer(dataset.median().select('NBR'), nbr_vis, 'NBR')
    m.add_layer(dataset.median(), rgb_vis, 'RGB')

    # Add layer control
    m.add_layer_control()  # This automatically places it at the top right

    return m

# Call the function to display the map
random_map = random_location_map()
random_map


Map(center=[40.096879771115425, -153.43249642925514], controls=(WidgetControl(options=['position', 'transparen…

Authenticate into GEE

In [4]:


def create_timelapse(lon, lat, start_date, end_date, export_path, frame_per_second=10):
    """Creates a timelapse GIF for the given coordinates and time range."""
    # Define thFailed to load model class 'LeafletMapModel' from module 'jupyter-leaflet'

    point = ee.Geometry.Point([lon, lat])
    region = point.buffer(10000).bounds()  # Buffer the point to create an area around the coordinates
    
    # Load Sentinel-2 image collection
    collection = (ee.ImageCollection('COPERNICUS/S2')
                  .filterBounds(point)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))  # Filter to reduce cloud cover
                  .select(['B4', 'B3', 'B2']))  # Select RGB bands for true color images

    # Prepare visualization parameters
    vis_params = {
        'min': 0,
        'max': 3000,
        'bands': ['B4', 'B3', 'B2']
    }

    # Set GIF arguments
    video_args = {
        'region': region,
        'dimensions': 768,
        'crs': 'EPSG:3857',
        'framesPerSecond': frame_per_second,
        'min': 0,
        'max': 3000,
        'format': 'gif'
    }

    # Generate and download the GIF
    geemap.download_ee_video(collection, video_args, export_path)
    
    # Example usage
lon = -117.1825
lat = 34.0556
start_date = '2019-01-01'
end_date = '2019-12-31'
export_path = '/path/to/export/timelapse.gif'
create_timelapse(lon, lat, start_date, end_date, export_path)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\path\to\export\timelapse.gif


In [22]:
Map = geemap.Map()
Map

roi = Map.user_roi
if roi is None:
    roi = ee.Geometry.BBox(117.1132, 3.5227, 117.2214, 3.5843)
    Map.addLayer(roi)
    Map.centerObject(roi)

timelapse = geemap.sentinel1_timelapse(
    roi,
    out_gif="sentinel1.gif",
    start_year=2023,
    end_year=2023,
    start_date="01-01",
    end_date="12-31",
    frequency="day",
    vis_params={"min": -30, "max": 0},
    palette="Greys",
    frames_per_second=1,
    title="Sentinel-1 Timelapse",
    add_colorbar=True,
    colorbar_bg_color="gray",
)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\brunolopez\machineLearning\geospatial\multispectral\sentinel1.gif


In [27]:
timelapse2 = geemap.sentinel2_timelapse(
    roi,
    out_gif="sentinel2.gif",
    start_year=2023,
    bands = ['Red', 'Green', 'Blue'],
    end_year=2023,
    start_date="01-01",
    apply_fmask = False,
    end_date="12-31",
    frequency="day",
    frames_per_second=1,
    title="Sentinel-2 Timelapse"
)

Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\brunolopez\machineLearning\geospatial\multispectral\sentinel2.gif


In [32]:
import imageio
from PIL import Image

def merge_gifs(gif1_path, gif2_path, output_path):
    # Load the two GIFs
    gif1 = imageio.mimread(gif1_path)
    gif2 = imageio.mimread(gif2_path)
    
    # Assume both GIFs have the same number of frames and same duration settings
    new_frames = []
    for img1, img2 in zip(gif1, gif2):
        # Convert to PIL images to stack
        img1_pil = Image.fromarray(img1)
        img2_pil = Image.fromarray(img2)
        
        # Get dimensions
        width1, height1 = img1_pil.size
        width2, height2 = img2_pil.size
        
        # Ensure the images are the same width for a vertical stack
        new_width = max(width1, width2)
        
        # Resize images if they are not the same width
        if width1 != new_width:
            img1_pil = img1_pil.resize((new_width, height1))
        if width2 != new_width:
            img2_pil = img2_pil.resize((new_width, height2))
        
        # Create new image with appropriate height
        total_height = height1 + height2
        new_img = Image.new('RGB', (new_width, total_height))
        
        # Paste images into new image
        new_img.paste(img1_pil, (0, 0))
        new_img.paste(img2_pil, (0, height1))
        
        # Append to frames
        new_frames.append(new_img)
    
    # Save stacked images as a new GIF
    new_frames[0].save(output_path, save_all=True, append_images=new_frames[1:], loop=0, duration=0.6)


# Usage
merge_gifs(r'C:\Users\brunolopez\machineLearning\geospatial\multispectral\sentinel1.gif', r'C:\Users\brunolopez\machineLearning\geospatial\multispectral\sentinel2.gif', r'C:\Users\brunolopez\machineLearning\geospatial\multispectral\sentinel1_2.gif')


In [34]:
import ee
import geemap
import reverse_geocode

ee.Initialize()

class SentinelTimelapse:
    def __init__(self, roi):
        self.Map = geemap.Map()
        if roi is None:
            # Default ROI if none provided
            self.roi = ee.Geometry.BBox(117.1132, 3.5227, 117.2214, 3.5843)
        else:
            self.roi = roi

        try:
            self.Map.addLayer(self.roi, {}, "ROI")
            self.Map.centerObject(self.roi)
            roi_center = self.roi.centroid().coordinates().getInfo()
            roi_center = (roi_center[1], roi_center[0])  # Lat, Lon
            self.location = reverse_geocode.search([roi_center])[0]
        except ee.EEException as e:
            print(f"Error processing ROI: {e}")

    def create_timelapse_s1(self, out_gif, start_year=2023, end_year=2023, frequency='day', fps=1):
        """Creates a Sentinel-1 timelapse."""
        return geemap.sentinel1_timelapse(
            self.roi,
            out_gif=out_gif,
            start_year=start_year,
            end_year=end_year,
            start_date="01-01",
            end_date="12-31",
            frequency=frequency,
            vis_params={"min": -30, "max": 0},
            palette="Greys",
            frames_per_second=fps,
            title=f"Sentinel-1 Timelapse - {self.location['city']}, {self.location['country']}",
            add_colorbar=True,
            colorbar_bg_color="gray"
        )

    def create_timelapse_s2(self, out_gif, start_year=2023, end_year=2023, bands=['Red', 'Green', 'Blue'], frequency='day', fps=1):
        """Creates a Sentinel-2 timelapse."""
        return geemap.sentinel2_timelapse(
            self.roi,
            out_gif=out_gif,
            start_year=start_year,
            bands=bands,
            end_year=end_year,
            start_date="01-01",
            apply_fmask=False,
            end_date="12-31",
            frequency=frequency,
            frames_per_second=fps,
            title=f"Sentinel-2 Timelapse - {self.location['city']}, {self.location['country']}"
        )

# Usage example
roi = ee.Geometry.BBox(117.1132, 3.5227, 117.2214, 3.5843)  # Define ROI
timelapse_creator = SentinelTimelapse(roi)
s1_gif_path = "sentinel1.gif"
s2_gif_path = "sentinel2.gif"
timelapse_creator.create_timelapse_s1(s1_gif_path)
timelapse_creator.create_timelapse_s2(s2_gif_path)


Error processing ROI: Geometry.centroid: Unable to perform this geometry operation. Please specify a non-zero error margin.


AttributeError: 'SentinelTimelapse' object has no attribute 'location'

In [37]:
import ee
import geemap
import reverse_geocode

ee.Initialize()

class SentinelTimelapse:
    def __init__(self, roi=None):
        self.Map = geemap.Map()
        if roi is None:
            self.roi = ee.Geometry.BBox(117.1132, 3.5227, 117.2214, 3.5843)
        else:
            self.roi = roi

        self.Map.addLayer(self.roi, {}, "ROI")
        self.Map.centerObject(self.roi)
        self.set_location()

    def set_location(self):
        try:
            roi_center = self.roi.centroid().coordinates().getInfo()
            roi_center = (roi_center[1], roi_center[0])  # Latitude, Longitude
            self.location = reverse_geocode.search([roi_center])[0]
            print(f"Location set to {self.location}")
        except Exception as e:
            self.location = {'city': 'Unknown', 'country': 'Unknown'}
            print(f"Failed to set location due to: {e}")

    def create_timelapse_s1(self, out_gif, start_year=2023, end_year=2023, frequency='day', fps=1):
        """Creates a Sentinel-1 timelapse."""
        return geemap.sentinel1_timelapse(
            self.roi,
            out_gif=out_gif,
            start_year=start_year,
            end_year=end_year,
            start_date="01-01",
            end_date="12-31",
            frequency=frequency,
            vis_params={"min": -30, "max": 0},
            palette="Greys",
            frames_per_second=fps,
            title=f"Sentinel-1 Timelapse - {self.location['city']}, {self.location['country']}",
            add_colorbar=True,
            colorbar_bg_color="gray"
        )

    def create_timelapse_s2(self, out_gif, start_year=2023, end_year=2023, bands=['Red', 'Green', 'Blue'], frequency='day', fps=1):
        """Creates a Sentinel-2 timelapse."""
        return geemap.sentinel2_timelapse(
            self.roi,
            out_gif=out_gif,
            start_year=start_year,
            bands=bands,
            end_year=end_year,
            start_date="01-01",
            apply_fmask=False,
            end_date="12-31",
            frequency=frequency,
            frames_per_second=fps,
            title=f"Sentinel-2 Timelapse - {self.location['city']}, {self.location['country']}"
        )

# Example usage
timelapse_creator = SentinelTimelapse()
s1_gif_path = "sentinel1.gif"
s2_gif_path = "sentinel2.gif"
timelapse_creator.create_timelapse_s1(s1_gif_path)
timelapse_creator.create_timelapse_s2(s2_gif_path)


Failed to set location due to: Geometry.centroid: Unable to perform this geometry operation. Please specify a non-zero error margin.
Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\brunolopez\machineLearning\geospatial\multispectral\sentinel1.gif
Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\brunolopez\machineLearning\geospatial\multispectral\sentinel2.gif


'c:\\Users\\brunolopez\\machineLearning\\geospatial\\multispectral\\sentinel2.gif'

In [38]:
import ee
import geemap
import reverse_geocode

ee.Initialize()

class SentinelTimelapse:
    def __init__(self, roi=None):
        self.Map = geemap.Map()
        if roi is None:
            # Define a default bounding box if none provided
            self.roi = ee.Geometry.BBox(117.1132, 3.5227, 117.2214, 3.5843)
        else:
            self.roi = roi

        self.Map.addLayer(self.roi, {}, "ROI")
        self.Map.centerObject(self.roi)
        self.set_location()

    def set_location(self):
        try:
            # Use coordinates directly from the BBox for reverse geocoding
            coordinates = self.roi.centroid(1).coordinates().getInfo()  # adding error margin 1
            coordinates = (coordinates[1], coordinates[0])  # Latitude, Longitude for reverse geocode
            self.location = reverse_geocode.search([coordinates])[0]
            print(f"Location set to {self.location}")
        except Exception as e:
            self.location = {'city': 'Unknown', 'country': 'Unknown'}
            print(f"Failed to set location due to: {e}")

    def create_timelapse_s1(self, out_gif, start_year=2023, end_year=2023, frequency='day', fps=1):
        """Creates a Sentinel-1 timelapse."""
        return geemap.sentinel1_timelapse(
            self.roi,
            out_gif=out_gif,
            start_year=start_year,
            end_year=end_year,
            start_date="01-01",
            end_date="12-31",
            frequency=frequency,
            vis_params={"min": -30, "max": 0},
            palette="Greys",
            frames_per_second=fps,
            title=f"Sentinel-1 Timelapse - {self.location['city']}, {self.location['country']}",
            add_colorbar=True,
            colorbar_bg_color="gray"
        )

    def create_timelapse_s2(self, out_gif, start_year=2023, end_year=2023, bands=['Red', 'Green', 'Blue'], frequency='day', fps=1):
        """Creates a Sentinel-2 timelapse."""
        return geemap.sentinel2_timelapse(
            self.roi,
            out_gif=out_gif,
            start_year=start_year,
            bands=bands,
            end_year=end_year,
            start_date="01-01",
            apply_fmask=False,
            end_date="12-31",
            frequency=frequency,
            frames_per_second=fps,
            title=f"Sentinel-2 Timelapse - {self.location['city']}, {self.location['country']}"
        )

# Example usage
timelapse_creator = SentinelTimelapse()
s1_gif_path = "sentinel1.gif"
s2_gif_path = "sentinel2.gif"
timelapse_creator.create_timelapse_s1(s1_gif_path)
timelapse_creator.create_timelapse_s2(s2_gif_path)


Location set to {'country_code': 'ID', 'city': 'Sesayap', 'country': 'Indonesia'}
Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\brunolopez\machineLearning\geospatial\multispectral\sentinel1.gif
Generating URL...
Please wait ...
The GIF image has been saved to: c:\Users\brunolopez\machineLearning\geospatial\multispectral\sentinel2.gif


'c:\\Users\\brunolopez\\machineLearning\\geospatial\\multispectral\\sentinel2.gif'